In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.0`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

import org.apache.spark.sql._
import org.apache.spark.sql.functions._

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("UDF")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
24/06/13 14:48:08 INFO SparkContext: Running Spark version 3.5.0
24/06/13 14:48:08 INFO SparkContext: OS info Linux, 6.5.0-35-generic, amd64
24/06/13 14:48:08 INFO SparkContext: Java version 11.0.23
24/06/13 14:48:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/13 14:48:08 INFO ResourceUtils: ==============================================================
24/06/13 14:48:08 INFO ResourceUtils: No custom resources configured for spark.driver.
24/06/13 14:48:08 INFO ResourceUtils: ==============================================================
24/06/13 14:48:08 INFO SparkContext: Submitted application: UDF
24/06/13 14:48:08 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name:

spark: SparkSession = org.apache.spark.sql.SparkSession@68741d27
import spark.implicits._

In [4]:
val data = Seq(
    (1, "Michael", 3000),
    (2, "Andy", 4500),
    (3, "Justin", 3500),
    (4, "Berta", 4000)
).toDF("id", "name", "salary")

data.show()

24/06/13 14:48:34 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/06/13 14:48:34 INFO SharedState: Warehouse path is 'file:/home/vadim/workspace/Spark/UDF%20UDAF/Notebooks/spark-warehouse'.
24/06/13 14:48:35 INFO CodeGenerator: Code generated in 200.82075 ms
24/06/13 14:48:36 INFO CodeGenerator: Code generated in 7.397915 ms
24/06/13 14:48:36 INFO CodeGenerator: Code generated in 15.811144 ms


+---+-------+------+
| id|   name|salary|
+---+-------+------+
|  1|Michael|  3000|
|  2|   Andy|  4500|
|  3| Justin|  3500|
|  4|  Berta|  4000|
+---+-------+------+



data: DataFrame = [id: int, name: string ... 1 more field]

## UDF без аргументов

In [5]:
val random = udf(() => java.lang.Math.random())
data.withColumn("random", random()).show()

24/06/13 14:48:43 INFO CodeGenerator: Code generated in 6.966186 ms
24/06/13 14:48:43 INFO CodeGenerator: Code generated in 7.891902 ms
24/06/13 14:48:43 INFO CodeGenerator: Code generated in 8.423446 ms


+---+-------+------+-------------------+
| id|   name|salary|             random|
+---+-------+------+-------------------+
|  1|Michael|  3000|   0.48753681279465|
|  2|   Andy|  4500|0.02202157680828698|
|  3| Justin|  3500|0.27643755299632966|
|  4|  Berta|  4000| 0.7762196368114738|
+---+-------+------+-------------------+



random: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell5$Helper$$Lambda$5029/0x00000008418e7c40@7f84a19d,
  DoubleType,
  List(),
  Some(
    ExpressionEncoder(
      BoundReference(0, DoubleType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, DoubleType),
          DoubleType,
          List("- root class: \"double\"")
        ),
        List("- root class: \"double\"")
      ),
      Double
    )
  ),
  None,
  false,
  true
)

## UDF с одним аргументом

In [6]:
val plusOne = udf((x: Int) => x + 1)
data.select($"name", $"salary", plusOne($"salary").as("new_salary")).show()

24/06/13 14:48:47 INFO CodeGenerator: Code generated in 7.359979 ms


+-------+------+----------+
|   name|salary|new_salary|
+-------+------+----------+
|Michael|  3000|      3001|
|   Andy|  4500|      4501|
| Justin|  3500|      3501|
|  Berta|  4000|      4001|
+-------+------+----------+



plusOne: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell6$Helper$$Lambda$5107/0x0000000841914040@b032378,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, IntegerType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, IntegerType),
          IntegerType,
          List("- root class: \"int\"")
        ),
        List("- root class: \"int\"")
      ),
      Int
    )
  ),
  None,
  false,
  true
)

## UDF с двумя аргументами

In [7]:
val strLenPlus = udf((x: String, y: Int) => x.length + y)

strLenPlus: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell7$Helper$$Lambda$5145/0x0000000841932840@7dcd9058,
  IntegerType,
  List(
    Some(
      ExpressionEncoder(
        StaticInvoke(
          class org.apache.spark.unsafe.types.UTF8String,
          StringType,
          "fromString",
          List(BoundReference(0, ObjectType(class java.lang.String), true)),
          List(),
          true,
          false,
          true
        ),
        Invoke(
          UpCast(
            GetColumnByOrdinal(0, StringType),
            StringType,
            List("- root class: \"java.lang.String\"")
          ),
          "toString",
          ObjectType(class java.lang.String),
          List(),
          List(),
          true,
          false,
          true
        ),
        java.lang.String
      )
    ),
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColu

In [8]:
data.select($"name", length($"name"), strLenPlus($"name",  $"id").as("nameLenPlusId")).show()

24/06/13 14:49:15 INFO CodeGenerator: Code generated in 5.580641 ms


+-------+------------+-------------+
|   name|length(name)|nameLenPlusId|
+-------+------------+-------------+
|Michael|           7|            8|
|   Andy|           4|            6|
| Justin|           6|            9|
|  Berta|           5|            9|
+-------+------------+-------------+



In [9]:
data.select($"name", length($"name"), strLenPlus($"name", lit(1)).as("nameLenPlus1")).show()

+-------+------------+------------+
|   name|length(name)|nameLenPlus1|
+-------+------------+------------+
|Michael|           7|           8|
|   Andy|           4|           5|
| Justin|           6|           7|
|  Berta|           5|           6|
+-------+------------+------------+



## UDF для выражений WHERE

In [10]:
val oneArgFilter= udf((n: Int) => { n > 2 })
data.where(oneArgFilter($"id")).show()

24/06/13 14:49:29 INFO CodeGenerator: Code generated in 22.82836 ms


+---+------+------+
| id|  name|salary|
+---+------+------+
|  3|Justin|  3500|
|  4| Berta|  4000|
+---+------+------+



oneArgFilter: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell10$Helper$$Lambda$5174/0x000000084193e840@66e69480,
  BooleanType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, IntegerType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, IntegerType),
            IntegerType,
            List("- root class: \"int\"")
          ),
          List("- root class: \"int\"")
        ),
        Int
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, BooleanType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, BooleanType),
          BooleanType,
          List("- root class: \"boolean\"")
        ),
        List("- root class: \"boolean\"")
      ),
      Boolean
    )
  ),
  None,
  false,
  true
)

## Производительность

In [11]:
val train = spark.read.option("inferSchema", true).option("header", true).csv("../data/train.csv").cache()

24/06/13 14:49:45 INFO InMemoryFileIndex: It took 28 ms to list leaf files for 1 paths.
24/06/13 14:49:45 INFO InMemoryFileIndex: It took 1 ms to list leaf files for 1 paths.
24/06/13 14:49:45 INFO FileSourceStrategy: Pushed Filters: 
24/06/13 14:49:45 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#143, None)) > 0)
24/06/13 14:49:46 INFO CodeGenerator: Code generated in 9.249635 ms
24/06/13 14:49:46 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 198.7 KiB, free 4.5 GiB)
24/06/13 14:49:46 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 4.5 GiB)
24/06/13 14:49:46 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on ubuntu:38097 (size: 34.1 KiB, free: 4.5 GiB)
24/06/13 14:49:46 INFO SparkContext: Created broadcast 0 from csv at cell11.sc:1
24/06/13 14:49:46 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304

train: Dataset[Row] = [row_id: int, timestamp: bigint ... 8 more fields]

In [12]:
train.show(5)

24/06/13 14:50:48 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 198.6 KiB, free 4.5 GiB)
24/06/13 14:50:48 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 34.1 KiB, free 4.5 GiB)
24/06/13 14:50:48 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on ubuntu:38097 (size: 34.1 KiB, free: 4.5 GiB)
24/06/13 14:50:48 INFO SparkContext: Created broadcast 4 from show at cell12.sc:1
24/06/13 14:50:48 INFO FileSourceScanExec: Planning scan with bin packing, max size: 134217728 bytes, open cost is considered as scanning 4194304 bytes.
24/06/13 14:50:48 INFO DAGScheduler: Got job 2 (show at cell12.sc:1) with 44 output partitions
24/06/13 14:50:48 INFO DAGScheduler: Final stage: ResultStage 2 (show at cell12.sc:1)
24/06/13 14:50:48 INFO DAGScheduler: Parents of final stage: List()
24/06/13 14:50:48 INFO DAGScheduler: Missing parents: List()
24/06/13 14:50:48 INFO DAGScheduler: Submitting ResultStage 2 (FileScan csv [row_i

+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|row_id|timestamp|user_id|content_id|content_type_id|task_container_id|user_answer|answered_correctly|prior_question_elapsed_time|prior_question_had_explanation|
+------+---------+-------+----------+---------------+-----------------+-----------+------------------+---------------------------+------------------------------+
|     0|        0|    115|      5692|              0|                1|          3|                 1|                       NULL|                          NULL|
|     1|    56943|    115|      5716|              0|                2|          2|                 1|                    37000.0|                         false|
|     2|   118363|    115|       128|              0|                0|          0|                 1|                    55000.0|                         false|
|     3|   131167|    115|  

In [13]:
train.count()

24/06/13 14:52:54 INFO CodeGenerator: Code generated in 5.930833 ms
24/06/13 14:52:54 INFO DAGScheduler: Registering RDD 23 (count at cell13.sc:1) as input to shuffle 0
24/06/13 14:52:54 INFO DAGScheduler: Got map stage job 4 (count at cell13.sc:1) with 44 output partitions
24/06/13 14:52:54 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (count at cell13.sc:1)
24/06/13 14:52:54 INFO DAGScheduler: Parents of final stage: List()
24/06/13 14:52:54 INFO DAGScheduler: Missing parents: List()
24/06/13 14:52:54 INFO DAGScheduler: Submitting ShuffleMapStage 4 (MapPartitionsRDD[23] at count at cell13.sc:1), which has no missing parents
24/06/13 14:52:54 INFO MemoryStore: Block broadcast_7 stored as values in memory (estimated size 23.7 KiB, free 2.2 GiB)
24/06/13 14:52:54 INFO MemoryStore: Block broadcast_7_piece0 stored as bytes in memory (estimated size 11.0 KiB, free 2.2 GiB)
24/06/13 14:52:54 INFO BlockManagerInfo: Added broadcast_7_piece0 in memory on ubuntu:38097 (size: 11.0 KiB, free:

res13: Long = 101230332L

In [14]:
val to_month = udf((ts_ms: Long) => ts_ms / 31536000000L * 12) // 1 year is 31536000000 ms

to_month: expressions.UserDefinedFunction = SparkUserDefinedFunction(
  ammonite.$sess.cell14$Helper$$Lambda$7208/0x0000000841fc0840@45cd211c,
  LongType,
  List(
    Some(
      ExpressionEncoder(
        BoundReference(0, LongType, false),
        AssertNotNull(
          UpCast(
            GetColumnByOrdinal(0, LongType),
            LongType,
            List("- root class: \"long\"")
          ),
          List("- root class: \"long\"")
        ),
        Long
      )
    )
  ),
  Some(
    ExpressionEncoder(
      BoundReference(0, LongType, false),
      AssertNotNull(
        UpCast(
          GetColumnByOrdinal(0, LongType),
          LongType,
          List("- root class: \"long\"")
        ),
        List("- root class: \"long\"")
      ),
      Long
    )
  ),
  None,
  false,
  true
)

In [15]:
val t1 = System.nanoTime

train
    .select($"content_id", to_month($"timestamp").as("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration1 = (System.nanoTime - t1) / 1e9d

24/06/13 14:53:36 INFO CodeGenerator: Code generated in 38.431156 ms
24/06/13 14:53:36 INFO DAGScheduler: Registering RDD 31 (show at cell15.sc:6) as input to shuffle 1
24/06/13 14:53:36 INFO DAGScheduler: Got map stage job 6 (show at cell15.sc:6) with 44 output partitions
24/06/13 14:53:36 INFO DAGScheduler: Final stage: ShuffleMapStage 7 (show at cell15.sc:6)
24/06/13 14:53:36 INFO DAGScheduler: Parents of final stage: List()
24/06/13 14:53:36 INFO DAGScheduler: Missing parents: List()
24/06/13 14:53:36 INFO DAGScheduler: Submitting ShuffleMapStage 7 (MapPartitionsRDD[31] at show at cell15.sc:6), which has no missing parents
24/06/13 14:53:36 INFO MemoryStore: Block broadcast_9 stored as values in memory (estimated size 54.6 KiB, free 2.2 GiB)
24/06/13 14:53:36 INFO MemoryStore: Block broadcast_9_piece0 stored as bytes in memory (estimated size 24.4 KiB, free 2.2 GiB)
24/06/13 14:53:36 INFO BlockManagerInfo: Added broadcast_9_piece0 in memory on ubuntu:38097 (size: 24.4 KiB, free: 2.

+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|0.4460020768431983|
|      4818|0.5068813295247988|
|      5518|0.6536082474226804|
|     13285| 2.725925925925926|
|     12027|2.2696177062374248|
|       833|0.7334516865330967|
|      9427| 0.652203699474766|
|       496|0.6270834569072898|
|      5156|0.5233297393761386|
|      6336|0.6127002288329519|
|      2866|0.9061162849232318|
|      1959|1.0353606789250354|
|      7982|0.6966064981949458|
|     23336| 0.564428149870339|
|       148| 0.602506180206291|
|      1342|0.7789912124752375|
|      1088|0.7588338484671289|
|      1580|0.7253544345532477|
|       471|0.8491909385113269|
|      2122|0.8964813870474247|
+----------+------------------+
only showing top 20 rows



t1: Long = 847252016840486L
duration1: Double = 4.413073671

In [16]:
val t2 = System.nanoTime

train
    .select($"content_id", ($"timestamp" / 31536000000L * 12).alias("month"))
    .groupBy("content_id")
    .mean("month")
    .show()

val duration2 = (System.nanoTime - t2) / 1e9d

24/06/13 14:53:58 INFO CodeGenerator: Code generated in 15.527991 ms
24/06/13 14:53:58 INFO DAGScheduler: Registering RDD 39 (show at cell16.sc:6) as input to shuffle 2
24/06/13 14:53:58 INFO DAGScheduler: Got map stage job 8 (show at cell16.sc:6) with 44 output partitions
24/06/13 14:53:58 INFO DAGScheduler: Final stage: ShuffleMapStage 10 (show at cell16.sc:6)
24/06/13 14:53:58 INFO DAGScheduler: Parents of final stage: List()
24/06/13 14:53:58 INFO DAGScheduler: Missing parents: List()
24/06/13 14:53:58 INFO DAGScheduler: Submitting ShuffleMapStage 10 (MapPartitionsRDD[39] at show at cell16.sc:6), which has no missing parents
24/06/13 14:53:58 INFO MemoryStore: Block broadcast_11 stored as values in memory (estimated size 49.5 KiB, free 2.2 GiB)
24/06/13 14:53:58 INFO MemoryStore: Block broadcast_11_piece0 stored as bytes in memory (estimated size 22.2 KiB, free 2.2 GiB)
24/06/13 14:53:58 INFO BlockManagerInfo: Added broadcast_11_piece0 in memory on ubuntu:38097 (size: 22.2 KiB, fre

+----------+------------------+
|content_id|        avg(month)|
+----------+------------------+
|      4519|2.2218736221822732|
|      4818|2.3372134952525556|
|      5518| 3.029691836679273|
|     13285| 5.964229418381344|
|     12027|5.9773868010962765|
|       833|  3.25216382900438|
|      9427| 3.108803192689145|
|       496|2.9023706505861853|
|      5156|2.6012427887203198|
|      6336|2.7640873674707946|
|      2866|3.8766860651262456|
|      1959|3.8461177532795423|
|      7982|3.0568602514634398|
|     23336| 2.773484159471813|
|       148| 2.579153999078694|
|      1342|  3.16126199595755|
|      1088| 3.545207472523368|
|      1580|3.2343945900405027|
|       471| 3.760585843300882|
|      2122|3.7522109521798543|
+----------+------------------+
only showing top 20 rows



t2: Long = 847273628762481L
duration2: Double = 2.772960518

In [17]:
println(duration1, duration2)

(4.413073671,2.772960518)
